<h1 style="color:#3da1da;font-size: 300%;" id="timeshap tutorial" align="center"  >TimeSHAP Tutorial - Pytorch - AReM dataset</h1><p>&nbsp;

<a id='top_cell'></a>

## Table of contents
1. [Data Processing](#1.-Data-Processing)
  1. [Data Loading](#1.1-Data-Loading)
  2. [Data Treatment](#1.2-Data-Treatment)
2. [Model](#2.-Model)
  1. [Model Definition](#2.1-Model-Definition)
  2. [Model Training](#2.2-Model-Training)
3. [TimeSHAP](#3.-TimeSHAP)
  1. [Local Explanations](#3.1-Local-Explanations)
  2. [Global Explanations](#3.2-Global-Explanations)
  3. [Individual Plots](#3.3-Individual-Plots)
    

# TimeSHAP

TimeSHAP is a model-agnostic, recurrent explainer that builds upon KernelSHAP and extends it to the sequential domain. 

TimeSHAP computes local event/timestamp- feature-, and cell-level attributions. 
    
Aditionally TimeSHAP also computes global event- and feature-level explanations.
    
As sequences can be arbitrarily long, TimeSHAP also implements a pruning algorithm based on Shapley Values, 
that finds a subset of consecutive, recent events that contribute the most to the decision.

---
# 1. Data-Processing
---

In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from timeshap import __version__
__version__

'1.0.1'

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import altair as alt
alt.themes.enable("feedzai")

ThemeRegistry.enable('feedzai')

## 1.1 Data-Loading

In [4]:
import os
import re

data_directories = next(os.walk("./AReM"))[1]

all_csvs = []
for folder in data_directories:
    if folder in ['bending1', 'bending2']:
        continue
    folder_csvs = next(os.walk(f"./AReM/{folder}"))[2]
    for data_csv in folder_csvs:
        if data_csv == 'dataset8.csv' and folder == 'sitting':
            # this dataset only has 479 instances
            # it is possible to use it, but would require padding logic
            continue
        loaded_data = pd.read_csv(f"./AReM/{folder}/{data_csv}", skiprows=4)
        print(f"{folder}/{data_csv} ------ {loaded_data.shape}")
        
        csv_id = re.findall(r'\d+', data_csv)[0]
        loaded_data['id'] = csv_id
        loaded_data['all_id'] = f"{folder}_{csv_id}"
        loaded_data['activity'] = folder
        all_csvs.append(loaded_data)

all_data = pd.concat(all_csvs)
raw_model_features = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
all_data.columns = ['timestamp', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23', 'id', 'all_id', 'activity']

sitting/dataset5.csv ------ (480, 7)
sitting/dataset11.csv ------ (480, 7)
sitting/dataset3.csv ------ (480, 7)
sitting/dataset10.csv ------ (480, 7)
sitting/dataset12.csv ------ (480, 7)
sitting/dataset4.csv ------ (480, 7)
sitting/dataset13.csv ------ (480, 7)
sitting/dataset9.csv ------ (480, 7)
sitting/dataset2.csv ------ (480, 7)
sitting/dataset15.csv ------ (480, 7)
sitting/dataset1.csv ------ (480, 7)
sitting/dataset14.csv ------ (480, 7)
sitting/dataset6.csv ------ (480, 7)
sitting/dataset7.csv ------ (480, 7)
standing/dataset5.csv ------ (480, 7)
standing/dataset11.csv ------ (480, 7)
standing/dataset3.csv ------ (480, 7)
standing/dataset8.csv ------ (480, 7)
standing/dataset10.csv ------ (480, 7)
standing/dataset12.csv ------ (480, 7)
standing/dataset4.csv ------ (480, 7)
standing/dataset13.csv ------ (480, 7)
standing/dataset9.csv ------ (480, 7)
standing/dataset2.csv ------ (480, 7)
standing/dataset15.csv ------ (480, 7)
standing/dataset1.csv ------ (480, 7)
standing/datase

## 1.2 Data Treatment

### Separate in train and test

In [5]:
# choose ids to use for test
ids_for_test = np.random.choice(all_data['id'].unique(), size=4, replace=False)

d_train =  all_data[~all_data['id'].isin(ids_for_test)]
d_test = all_data[all_data['id'].isin(ids_for_test)]

###  Normalize Features

In [6]:
class NumericalNormalizer:
    def __init__(self, fields: list):
        self.metrics = {}
        self.fields = fields

    def fit(self, df: pd.DataFrame ) -> list:
        means = df[self.fields].mean()
        std = df[self.fields].std()
        for field in self.fields:
            field_mean = means[field]
            field_stddev = std[field]
            self.metrics[field] = {'mean': field_mean, 'std': field_stddev}

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        # Transform to zero-mean and unit variance.
        for field in self.fields:
            f_mean = self.metrics[field]['mean']
            f_stddev = self.metrics[field]['std']
            # OUTLIER CLIPPING to [avg-3*std, avg+3*avg]
            df[field] = df[field].apply(lambda x: f_mean - 3 * f_stddev if x < f_mean - 3 * f_stddev else x)
            df[field] = df[field].apply(lambda x: f_mean + 3 * f_stddev if x > f_mean + 3 * f_stddev else x)
            if f_stddev > 1e-5:
                df[f'p_{field}_normalized'] = df[field].apply(lambda x: ((x - f_mean)/f_stddev))
            else:
                df[f'p_{field}_normalized'] = df[field].apply(lambda x: x * 0)
        return df

In [7]:
#all features are numerical
normalizor = NumericalNormalizer(raw_model_features)
normalizor.fit(d_train)
d_train_normalized = normalizor.transform(d_train)
d_test_normalized = normalizor.transform(d_test)

### Features

In [8]:
model_features = [f"p_{x}_normalized" for x in raw_model_features]
time_feat = 'timestamp'
label_feat = 'activity'
sequence_id_feat = 'all_id'

plot_feats = {
    'p_avg_rss12_normalized': "Mean Chest <-> Right Ankle",
    'p_var_rss12_normalized': "STD Chest <-> Right Ankle",
    'p_avg_rss13_normalized': "Mean Chest <-> Left Ankle",
    'p_var_rss13_normalized': "STD Chest <-> Left Ankle",
    'p_avg_rss23_normalized': "Mean Right Ankle <-> Left Ankle",
    'p_var_rss23_normalized': "STD Right Ankle <-> Left Ankle",
}

### Transform the dataset from multi-label to binary classification

In [9]:
# possible activities ['cycling', 'lying', 'sitting', 'standing', 'walking']
#Select the activity to predict
chosen_activity = 'cycling'

d_train_normalized['label'] = d_train_normalized['activity'].apply(lambda x: int(x == chosen_activity))
d_test_normalized['label'] = d_test_normalized['activity'].apply(lambda x: int(x == chosen_activity))

This example notebook requires PyTorch!

Install it if you haven't already:
```
!pip install torch
```

In [10]:
import torch

def df_to_Tensor(df, model_feats, label_feat, group_by_feat, timestamp_Feat):
    sequence_length = len(df[timestamp_Feat].unique())
    
    data_tensor = np.zeros((len(df[group_by_feat].unique()), sequence_length, len(model_feats)))
    labels_tensor = np.zeros((len(df[group_by_feat].unique()), 1))
    
    for i, name in enumerate(df[group_by_feat].unique()):
        name_data = df[df[group_by_feat] == name]
        sorted_data = name_data.sort_values(timestamp_Feat)
        
        data_x = sorted_data[model_feats].values
        labels = sorted_data[label_feat].values
        assert labels.sum() == 0 or labels.sum() == len(labels)
        data_tensor[i, :, :] = data_x
        labels_tensor[i, :] = labels[0]
    data_tensor = torch.from_numpy(data_tensor).type(torch.FloatTensor)
    labels_tensor = torch.from_numpy(labels_tensor).type(torch.FloatTensor)
    
    return data_tensor, labels_tensor

In [11]:
train_data, train_labels = df_to_Tensor(d_train_normalized, model_features, 'label', sequence_id_feat, time_feat)

test_data, test_labels = df_to_Tensor(d_test_normalized, model_features, 'label', sequence_id_feat, time_feat)

___
# 2. Model


## 2.1 Model Definition

In [12]:
import torch.nn as nn

class ExplainedRNN(nn.Module):
    def __init__(self,
                 input_size: int,
                 cfg: dict,
                 ):
        super(ExplainedRNN, self).__init__()
        self.hidden_dim = cfg.get('hidden_dim', 32)
        torch.manual_seed(cfg.get('random_seed', 42))

        self.recurrent_block = nn.GRU(
            input_size=input_size,
            hidden_size=self.hidden_dim,
            batch_first=True,
            num_layers=2,
            )
        
        self.classifier_block = nn.Linear(self.hidden_dim, 1)
        self.output_activation_func = nn.Sigmoid()

    def forward(self,
                x: torch.Tensor,
                hidden_states: tuple = None,
                ):

        if hidden_states is None:
            output, hidden = self.recurrent_block(x)
        else:
            output, hidden = self.recurrent_block(x, hidden_states)

        # -1 on hidden, to select the last layer of the stacked gru
        assert torch.equal(output[:,-1,:], hidden[-1, :, :])
        
        y = self.classifier_block(hidden[-1, :, :])
        y = self.output_activation_func(y)
        return y, hidden

In [13]:
import torch.optim as optim

model = ExplainedRNN(len(model_features), {})
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

learning_rate = 0.005
EPOCHS = 8

## 2.2 Model Training

In [14]:
import tqdm
import copy
for epoch in tqdm.trange(EPOCHS):
    train_data_local = copy.deepcopy(train_data)
    train_labels_local = copy.deepcopy(train_labels)
    
    y_pred, hidden_states = model(train_data_local)
    train_loss = loss_function(y_pred, train_labels_local)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    with torch.no_grad():
        test_data_local = copy.deepcopy(test_data)
        test_labels_local = copy.deepcopy(test_labels)
        test_preds, _ = model(test_data_local)
        test_loss = loss_function(test_preds, test_labels_local)
        print(f"Train loss: {train_loss.item()} --- Test loss {test_loss.item()} ")


 12%|█▎        | 1/8 [00:00<00:05,  1.18it/s]

Train loss: 0.6997937560081482 --- Test loss 0.5927574634552002 


 25%|██▌       | 2/8 [00:02<00:06,  1.05s/it]

Train loss: 0.5918577313423157 --- Test loss 0.4946227967739105 


 38%|███▊      | 3/8 [00:03<00:05,  1.01s/it]

...

Train loss: 0.334475576877594 --- Test loss 0.2376544028520584 


 88%|████████▊ | 7/8 [00:06<00:00,  1.13it/s]

Train loss: 0.29900506138801575 --- Test loss 0.1931639164686203 


100%|██████████| 8/8 [00:07<00:00,  1.09it/s]

Train loss: 0.25920480489730835 --- Test loss 0.17529389262199402 


---
# 3. TimeSHAP
---

### Model entry point

In [15]:
from timeshap.wrappers import TorchModelWrapper
model_wrapped = TorchModelWrapper(model)
f_hs = lambda x, y=None: model_wrapped.predict_last_hs(x, y)

### Baseline event

In [16]:
from timeshap.utils import calc_avg_event
average_event = calc_avg_event(d_train_normalized, numerical_feats=model_features, categorical_feats=[])

In [17]:
average_event

,p_avg_rss12_normalized,p_var_rss12_normalized,p_avg_rss13_normalized,p_var_rss13_normalized,p_avg_rss23_normalized,p_var_rss23_normalized
0,0.095791,-0.531224,0.178574,-0.416398,0.129324,-0.392722


### Baseline Sequence

In [18]:
from timeshap.utils import calc_avg_sequence
average_sequence = calc_avg_sequence(d_train_normalized, numerical_feats=model_features, categorical_feats=[],model_features=model_features, entity_col=sequence_id_feat)

In [19]:
average_sequence

array([[0.0000e+00, 4.0875e+01, 5.0000e-01, 1.4625e+01, 1.0500e+00,
        1.5250e+01],
       [2.5000e+02, 4.0585e+01, 6.0500e-01, 1.4875e+01, 7.9000e-01,
        1.4875e+01],
       [5.0000e+02, 3.9835e+01, 7.5500e-01, 1.4125e+01, 8.7000e-01,
        1.7000e+01],
       ...,
       [1.1925e+05, 3.9625e+01, 6.6000e-01, 1.4400e+01, 9.7000e-01,
        1.4165e+01],
       [1.1950e+05, 3.9500e+01, 5.0000e-01, 1.2710e+01, 1.2050e+00,
        1.5000e+01],
       [1.1975e+05, 3.9625e+01, 6.6000e-01, 1.4000e+01, 1.0150e+00,
        1.4750e+01]])

### Average score over baseline

In [20]:
from timeshap.utils import get_avg_score_with_avg_event
avg_score_over_len = get_avg_score_with_avg_event(f_hs, average_event, top=480)

## 3.1 Local Explanations

### Select sequences to explain

In [21]:
positive_sequence_id = f"cycling_{np.random.choice(ids_for_test)}"
pos_x_pd = d_test_normalized[d_test_normalized['all_id'] == positive_sequence_id]

# select model features only
pos_x_data = pos_x_pd[model_features]
# convert the instance to numpy so TimeSHAP receives it
pos_x_data = np.expand_dims(pos_x_data.to_numpy().copy(), axis=0)

### Local Report on positive instance

In [22]:
from timeshap.explainer import local_report

pruning_dict = {'tol': 0.025}
event_dict = {'rs': 42, 'nsamples': 32000}
feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_feats': 2, 'top_x_events': 2}
local_report(f_hs, pos_x_data, pruning_dict, event_dict, feature_dict, cell_dict, average_event, entity_uuid=positive_sequence_id, entity_col='all_id')

Assuming all features are model features


alt.HConcatChart(...)

## 3.2 Global Explanations

### Explain all 

TimeSHAP offers methods to explain all instances and save as CSV.
This allows for global explanations and local plots with no calculation delay.

In [23]:
from timeshap.explainer import global_report 

pos_dataset = d_test_normalized[d_test_normalized['label'] == 1]
schema = schema = list(pos_dataset.columns)
pruning_dict = {'tol': [0.05, 0.075], 'path': 'outputs/prun_all.csv'}
event_dict = {'path': 'outputs/event_all.csv', 'rs': 42, 'nsamples': 32000}
feature_dict = {'path': 'outputs/feature_all.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats,}
prun_stats, global_plot = global_report(f_hs, pos_dataset, pruning_dict, event_dict, feature_dict, average_event, model_features, schema, sequence_id_feat, time_feat)
prun_stats

Calculating pruning algorithm
Calculating event data
Calculating feat data
Calculating pruning indexes


,Tolerance,Mean,Std
0,0.05,12.75,1.258306
1,0.075,11.25,1.258306
2,No Pruning,480.00,0.000000


In [24]:
global_plot

alt.VConcatChart(...)

## 3.3 Individual Plots

### Local Plots

In [25]:
from timeshap.plot import plot_temp_coalition_pruning, plot_event_heatmap, plot_feat_barplot, plot_cell_level
from timeshap.explainer import local_pruning, local_event, local_feat, local_cell_level
# select model features only
pos_x_data = pos_x_pd[model_features]
# convert the instance to numpy so TimeSHAP receives it
pos_x_data = np.expand_dims(pos_x_data.to_numpy().copy(), axis=0)

##### Pruning algorithm

In [26]:
pruning_dict = {'tol': 0.025,}
coal_plot_data, coal_prun_idx = local_pruning(f_hs, pos_x_data, pruning_dict, average_event, positive_sequence_id, sequence_id_feat, False)
# coal_prun_idx is in negative terms
pruning_idx = pos_x_data.shape[1] + coal_prun_idx
pruning_plot = plot_temp_coalition_pruning(coal_plot_data, coal_prun_idx, plot_limit=40)
pruning_plot

alt.LayerChart(...)

##### Event-level explanation

In [27]:
event_dict = {'rs': 42, 'nsamples': 32000}
event_data = local_event(f_hs, pos_x_data, event_dict, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
event_plot = plot_event_heatmap(event_data)
event_plot

alt.LayerChart(...)

##### Feature-level explanation

In [28]:
feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
feature_data = local_feat(f_hs, pos_x_data, feature_dict, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
feature_plot = plot_feat_barplot(feature_data, feature_dict.get('top_feats'), feature_dict.get('plot_features'))
feature_plot

alt.LayerChart(...)

##### Cell-level explanation

In [29]:
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_events': 3, 'top_x_feats': 3}
cell_data = local_cell_level(f_hs, pos_x_data, cell_dict, event_data, feature_data, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
feat_names = list(feature_data['Feature'].values)[:-1] # exclude pruned events
cell_plot = plot_cell_level(cell_data, feat_names, feature_dict.get('plot_features'))
cell_plot

alt.HConcatChart(...)

### Global Plots

In [30]:
from timeshap.explainer import prune_all, pruning_statistics, event_explain_all, feat_explain_all
from timeshap.plot import plot_global_event, plot_global_feat

pos_dataset = d_test_normalized[d_test_normalized['label'] == 1]

##### Pruning statistics

In [31]:
pruning_dict = {'tol': [0.05, 0.075], 'path': 'outputs/prun_all.csv'}
prun_indexes = prune_all(f_hs, pos_dataset, pruning_dict, average_event, model_features, schema, sequence_id_feat, time_feat)
pruning_stats = pruning_statistics(prun_indexes, pruning_dict.get('tol'))
pruning_stats

,Tolerance,Mean,Std
0,0.05,12.75,1.258306
1,0.075,11.25,1.258306
2,No Pruning,480.00,0.000000


##### Global event-level

In [32]:
event_dict = {'path': 'outputs/event_all.csv', 'rs': 42, 'nsamples': 32000}
event_data = event_explain_all(f_hs, pos_dataset, event_dict, prun_indexes, average_event, model_features, schema, sequence_id_feat, time_feat)
event_global_plot = plot_global_event(event_data)
event_global_plot

alt.VConcatChart(...)

##### Global feature-level

In [33]:
feature_dict = {'path': 'outputs/feature_all.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats, }
feat_data = feat_explain_all(f_hs, pos_dataset, feature_dict, prun_indexes, average_event, model_features, schema, sequence_id_feat, time_feat)
feat_global_plot = plot_global_feat(feat_data, **feature_dict)
feat_global_plot

alt.VConcatChart(...)